In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import helper
api = DataLoader()

d:\Practice\stock_trade\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [25]:
def print_result(stats):
  stats = stats.loc[[
      'Start',
      'End',
      'Equity Final [$]',      # 最終資產淨值
      'Return [%]',            # 總報酬率
      'Buy & Hold Return [%]', # 買入並持有報酬率（基準）
      'Max. Drawdown [%]',     # 最大回撤
      '# Trades',
      'Win Rate [%]',          # 勝率
      'SQN',                    # 系統品質數字
      '_trades'
    ]]
  print('------------------回測表現-----------------')
  print(stats)
  print('------------------交易紀錄-----------------')
  trades = stats['_trades']
  for _, trade in trades.iterrows():
      print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
      print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

In [28]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(months=24)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '6770'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'max': 'High', 'min': 'Low', 'open': 'Open', 'close': 'Close', 'Trading_Volume': 'Volume'})
df

2025-10-21 11:29:19.491 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 6770


start date 2023-10-21
end date 2025-10-21


,stock_id,Volume,Trading_money,Open,High,Low,Close,spread,Trading_turnover
date,,,,,,,,,
2023-10-23,6770,11869236,318253249,27.05,27.10,26.65,26.65,-0.65,5911
2023-10-24,6770,9790215,260207070,26.55,26.90,26.25,26.85,0.20,4782
2023-10-25,6770,20642290,570142540,26.90,28.15,26.70,27.50,0.65,8706
2023-10-26,6770,11399649,308594379,27.15,27.45,26.80,26.80,-0.70,5895
2023-10-27,6770,6202142,165891222,26.90,27.00,26.60,26.65,-0.15,3069
...,...,...,...,...,...,...,...,...,...
2025-10-14,6770,294354305,8467231829,30.60,31.00,27.30,27.30,-1.95,102496
2025-10-15,6770,138777196,3867298262,28.35,28.45,27.40,27.80,0.50,52512
2025-10-16,6770,292728145,8611277025,28.50,30.25,28.45,28.80,1.00,94120


In [31]:
helper.EMA_KD.n1 = 10
helper.EMA_KD.n2 = 50
bt = Backtest(df, helper.EMA_KD, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)
# stats

Backtest.run:   0%|          | 0/476 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=156: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=308: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=385: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=394: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=400: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d

------------------回測表現-----------------
Start                                                  2023-10-23 00:00:00
End                                                    2025-10-20 00:00:00
Equity Final [$]                                             113688.901725
Return [%]                                                       13.688902
Buy & Hold Return [%]                                             9.782609
Max. Drawdown [%]                                               -57.007383
# Trades                                                                 1
Win Rate [%]                                                         100.0
SQN                                                                    NaN
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-05-23 00:00:00, Sell date: 2025-10-02 00:00:00
Buy price: 26.4, Sell price: 30.1, P/L: 13.71%


In [32]:
helper.EMA_VWAP_KD.n1 = 10
helper.EMA_VWAP_KD.n2 = 50
bt = Backtest(df, helper.EMA_VWAP_KD, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

------------------回測表現-----------------
Start                                                  2023-10-23 00:00:00
End                                                    2025-10-20 00:00:00
Equity Final [$]                                                  100000.0
Return [%]                                                             0.0
Buy & Hold Return [%]                                             9.782609
Max. Drawdown [%]                                                     -0.0
# Trades                                                                 0
Win Rate [%]                                                           NaN
SQN                                                                    NaN
_trades                  Empty DataFrame
Columns: [Size, EntryBar, Exit...
dtype: object
------------------交易紀錄-----------------


In [18]:
helper.SmaCross.n1 = 5
helper.SmaCross.n2 = 20
bt = Backtest(df, helper.SmaCross, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

------------------回測表現-----------------
Equity Final [$]                                             115646.485262
Return [%]                                                       15.646485
Buy & Hold Return [%]                                            -4.451039
Max. Drawdown [%]                                               -42.452524
Win Rate [%]                                                     30.769231
SQN                                                               0.271201
_trades                      Size  EntryBar  ExitBar  EntryPrice  ExitP...
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-01-08 00:00:00, Sell date: 2024-03-12 00:00:00
Buy price: 184.5, Sell price: 228.0, P/L: 23.26%
Buy date: 2024-05-02 00:00:00, Sell date: 2024-07-01 00:00:00
Buy price: 225.0, Sell price: 300.0, P/L: 33.00%
Buy date: 2024-08-23 00:00:00, Sell date: 2024-09-10 00:00:00
Buy price: 214.0, Sell price: 215.0, P/L: 0.18%
Buy date: 2024-09-23 00:00:00, Sell date: 2024-10

In [34]:
helper.SMA_KD.n1 = 10
helper.SMA_KD.n2 = 50
bt = Backtest(df, helper.SMA_KD, cash=30_000, commission=.001425)
stats = bt.run()
print_result(stats)

Backtest.run:   0%|          | 0/435 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=382: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=450: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=458: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                      

------------------回測表現-----------------
Start                                                  2023-10-23 00:00:00
End                                                    2025-10-20 00:00:00
Equity Final [$]                                              35318.614298
Return [%]                                                       17.728714
Buy & Hold Return [%]                                             2.886248
Max. Drawdown [%]                                               -56.999936
# Trades                                                                 2
Win Rate [%]                                                         100.0
SQN                                                               1.004241
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
dtype: object
------------------交易紀錄-----------------
Buy date: 2025-02-05 00:00:00, Sell date: 2025-10-02 00:00:00
Buy price: 18.8, Sell price: 30.1, P/L: 59.74%
Buy date: 2024-05-22 00:00:00, Sell date: 2025-

In [35]:
helper.BOLL_KD30.buy_strategy = helper.BuyStrategy.BOLL_KD30
bt = Backtest(df, helper.BOLL_KD30, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

Backtest.run:   0%|          | 0/465 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=63: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=103: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=104: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=109: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=110: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:

------------------回測表現-----------------
Start                                                  2023-10-23 00:00:00
End                                                    2025-10-20 00:00:00
Equity Final [$]                                              89638.887977
Return [%]                                                      -10.361112
Buy & Hold Return [%]                                             4.302926
Max. Drawdown [%]                                               -34.169337
# Trades                                                                 5
Win Rate [%]                                                          60.0
SQN                                                              -0.694743
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-01-18 00:00:00, Sell date: 2024-06-18 00:00:00
Buy price: 27.6, Sell price: 28.05, P/L: 1.34%
Buy date: 2024-07-18 00:00:00, Sell date: 2024-